======================== Import Packages ==========================

In [ ]:
import sys, pdb, glob
import numpy as np
from astropy.table import Table, join
from astroquery.vizier import Vizier
import warnings
from astropy.logger import AstropyWarning
warnings.filterwarnings('ignore', category=AstropyWarning)

===================== Define Functions ===================

In [ ]:
def get_data(catalog, join_key='Name'):

    """
    PURPOSE:    Get data from literature with Vizier

    INPUT:      catalog = ctalog name on Vizier (str)
                join_key = column header to join tables, if multiple (str; optional)

    OUTPUT:     t = data table (AstroPy Table)

    """

    ### GET FULL CATALOG (ALL COLUMNS, ALL ROWS)
    viz = Vizier(catalog=catalog, columns=['**'])
    viz.ROW_LIMIT = -1
    tv = viz.get_catalogs(catalog)

    ### IF MULTIPLE TABLES, JOIN THEN
    for i, val in enumerate(tv.keys()):
        if i == 0:
            t = tv[val]
        else:
            tt = tv[val]
            if join_key in tt.columns:
                t = join(t, tt, join_type='inner', keys=join_key)

    return t

========================== Code ==========================

In [ ]:
### LOAD IN LUPUS DATA
T = get_data("J/ApJ/828/46")
T.sort('RAJ2000')

In [ ]:
### WRITE HEADER INFO
f = open('../output/table_02.tex', 'w')
f.write(r'\capstartfalse'                                  + ' \n')
f.write(r'\begin{deluxetable*}{lrrrccccr}'                 + ' \n')
f.write(r'\tabletypesize{\footnotesize}'                   + ' \n')
f.write(r'\centering'                                      + ' \n')
f.write(r'\tablewidth{500pt}'                              + ' \n')
f.write(r'\tablecaption{$890~\mu$m Continuum Properties \label{tab-cont}}' + ' \n')
f.write(r'\tablecolumns{9} '                               + ' \n')
f.write(r'\tablehead{'                                     + ' \n')
f.write(r' \colhead{Source}'                               + ' \n')
f.write(r'&\colhead{RA$_{\rm J2000}$}'                     + ' \n')
f.write(r'&\colhead{Dec$_{\rm J2000}$}'                    + ' \n')
f.write(r'&\colhead{$F_{\rm cont}$}'                       + ' \n')
f.write(r'&\colhead{rms}'                                  + ' \n')
f.write(r'&\colhead{$a$}'                                  + ' \n')
f.write(r'&\colhead{$i$}'                                  + ' \n')
f.write(r'&\colhead{PA}'                                   + ' \n')
f.write(r'&\colhead{$M_{\rm dust}$} \\'                    + ' \n')
f.write(r' \colhead{}'                                     + ' \n')
f.write(r'&\colhead{}'                                     + ' \n')
f.write(r'&\colhead{}'                                     + ' \n')
f.write(r'&\colhead{(mJy)}'                                + ' \n')
f.write(r'&\colhead{(mJy beam$^{-1}$)}'                    + ' \n')
f.write(r'&\colhead{arcsec}'                               + ' \n')
f.write(r'&\colhead{deg}'                                  + ' \n')
f.write(r'&\colhead{(deg)}'                                + ' \n')
f.write(r'&\colhead{($M_{\oplus}$)}'                       + ' \n')
f.write(r'}'                                               + ' \n')
f.write(r'\startdata'                                      + ' \n')

In [ ]:
### WRITE FIRST 10 LINES
lim = 10
for i, val in enumerate(np.zeros(lim)):

    source = T['Name'][i]
    de =  T['DEJ2000'][i].replace(' ', ':')
    ra = T['RAJ2000'][i].replace(' ', ':')
    flx = "{0:.2f}".format(T['F890'][i]) + r' $\pm$ ' + "{0:.2f}".format(T['e_F890'][i])
    rms = "{0:.2f}".format(T['rms'][i])
    mdust = str(T['MDust'][i]) + r' $\pm$ ' + str(T['e_MDust'][i])

    if np.ma.is_masked(T['a'][i]) is True:
        bmaj = '...'
    else:
        bmaj  = str(T['a'][i]) + r' $\pm$ ' + str(T['e_a'][i])

    if np.ma.is_masked(T['i'][i]) is True:
        incl = '...'
    elif np.ma.is_masked(T['e_i'][i]) is True:
        incl = str(T['i'][i])
    else:
        incl  = str(T['i'][i]) + r' $\pm$ ' + str(T['e_i'][i])

    if np.ma.is_masked(T['PA'][i]) is True:
        pang = '...'
    elif np.ma.is_masked(T['e_PA'][i]) is True:
        pang = str(T['PA'][i])
    else:
        pang  = str(T['PA'][i]) + r' $\pm$ ' + str(T['e_PA'][i])

    ### END OF LINE
    if (i < lim - 1):
        end = r' \\' + '\n'
    else:
        end = '\n'

    f.write(source + ' & ' + ra + ' & ' + de + ' & ' + flx + ' & ' + rms + ' & ' + 
            bmaj + ' & ' + incl + ' & ' + pang + ' & ' + mdust + end)

In [ ]:
f.write(r'\enddata' + ' \n')
f.write(r'\tablenotetext{}{(This table is available in its entirety in machine-readible form.)}' + ' \n')
f.write(r'\end{deluxetable*}' + ' \n')
f.write(r'\capstartfalse' + ' \n')
f.close()